In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6106,2020-12-10T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1342,197,1539,35430,...,1895,NaN,NaN,75367,1056737,712959.0,NaN,21.0,NaN,NaN
6107,2020-12-10T17:00:00,ITA,9,Toscana,43.769231,11.255889,1323,247,1570,20762,...,3032,NaN,NaN,110440,1680849,993998.0,NaN,11.0,Positivi diagnosticati solo con test antigenic...,NaN
6108,2020-12-10T17:00:00,ITA,10,Umbria,43.106758,12.388247,310,55,365,5014,...,498,NaN,NaN,25615,447196,234249.0,NaN,5.0,NaN,NaN
6109,2020-12-10T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,98,10,108,675,...,344,NaN,NaN,6801,57189,34989.0,NaN,0.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6106,2020-12-10T17:00:00,19,Sicilia,1342,197,1539,35430,36969,-1678,1059,36503,1895,NaN,NaN,75367,1056737,712959.0,21.0,NaN,NaN
6107,2020-12-10T17:00:00,9,Toscana,1323,247,1570,20762,22332,-2273,517,85076,3032,NaN,NaN,110440,1680849,993998.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN
6108,2020-12-10T17:00:00,10,Umbria,310,55,365,5014,5379,-277,182,19738,498,NaN,NaN,25615,447196,234249.0,5.0,NaN,NaN
6109,2020-12-10T17:00:00,2,Valle d'Aosta,98,10,108,675,783,-73,18,5674,344,NaN,NaN,6801,57189,34989.0,0.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-12-10', '2020-12-09')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-10T17:00:00,1,Piemonte,4069,310,4379,56728,61107,-2376,974,113527,6897,NaN,NaN,181531,1712408,1018261.0,14.0,Il numero dei tamponi antigenici positivi è so...,NaN
1,2020-12-10T17:00:00,2,Valle d'Aosta,98,10,108,675,783,-73,18,5674,344,NaN,NaN,6801,57189,34989.0,0.0,NaN,NaN
2,2020-12-10T17:00:00,3,Lombardia,5613,748,6361,80882,87243,-2660,2093,324961,23449,NaN,NaN,435653,4350769,2436786.0,62.0,NaN,NaN
3,2020-12-10T17:00:00,5,Veneto,2606,330,2936,80201,83137,2119,4197,89880,4551,NaN,NaN,177568,2941145,1108040.0,42.0,NaN,NaN
4,2020-12-10T17:00:00,6,Friuli Venezia Giulia,652,62,714,13481,14195,-760,672,22430,1164,NaN,NaN,37789,792239,303968.0,5.0,NaN,NaN
5,2020-12-10T17:00:00,7,Liguria,822,85,907,8051,8958,-289,320,43124,2602,NaN,NaN,54684,637449,301691.0,3.0,i 104903 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-10T17:00:00,8,Emilia-Romagna,2845,229,3074,61643,64717,-1126,1453,68697,6386,NaN,NaN,139800,2276951,1133307.0,17.0,NaN,NaN
7,2020-12-10T17:00:00,9,Toscana,1323,247,1570,20762,22332,-2273,517,85076,3032,NaN,NaN,110440,1680849,993998.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-10T17:00:00,10,Umbria,310,55,365,5014,5379,-277,182,19738,498,NaN,NaN,25615,447196,234249.0,5.0,NaN,NaN
9,2020-12-10T17:00:00,11,Marche,501,86,587,12632,13219,-298,323,18791,1373,NaN,NaN,33383,443307,259267.0,5.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-09T17:00:00,1,Piemonte,4102,318,4420,59063,63483,-1030,906,110261,6813,NaN,NaN,180557,1697990,1012701.0,8.0,NaN,NaN
1,2020-12-09T17:00:00,2,Valle d'Aosta,90,13,103,753,856,-14,17,5584,343,NaN,NaN,6783,56847,34887.0,4.0,NaN,NaN
2,2020-12-09T17:00:00,3,Lombardia,5727,766,6493,83410,89903,-21926,1233,320380,23277,NaN,NaN,433560,4326540,2429884.0,24.0,NaN,NaN
3,2020-12-09T17:00:00,5,Veneto,2551,319,2870,78148,81018,1270,2427,87950,4403,NaN,NaN,173371,2921028,1101887.0,14.0,NaN,NaN
4,2020-12-09T17:00:00,6,Friuli Venezia Giulia,663,63,726,14229,14955,78,633,21040,1122,NaN,NaN,37117,783883,301832.0,5.0,NaN,NaN
5,2020-12-09T17:00:00,7,Liguria,841,88,929,8318,9247,-498,191,42526,2591,NaN,NaN,54364,633457,300174.0,4.0,i 99296 test antigenici rapidi sono riferiti a...,NaN
6,2020-12-09T17:00:00,8,Emilia-Romagna,2857,233,3090,62753,65843,-679,1079,66191,6314,NaN,NaN,138348,2259395,1128691.0,8.0,NaN,NaN
7,2020-12-09T17:00:00,9,Toscana,1347,253,1600,23005,24605,-1419,505,82361,2957,NaN,NaN,109923,1671272,990816.0,12.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-09T17:00:00,10,Umbria,323,56,379,5277,5656,-90,89,19291,486,NaN,NaN,25433,443349,233112.0,2.0,NaN,NaN
9,2020-12-09T17:00:00,11,Marche,518,85,603,12914,13517,-304,229,18179,1364,NaN,NaN,33060,440236,257377.0,2.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-10T17:00:00,1,Piemonte,4069,310,4379,56728,61107,-2376,974,113527,6897,NaN,NaN,181531,1712408,1018261.0,14.0,Il numero dei tamponi antigenici positivi è so...,NaN
1,2020-12-10T17:00:00,2,Valle d'Aosta,98,10,108,675,783,-73,18,5674,344,NaN,NaN,6801,57189,34989.0,0.0,NaN,NaN
2,2020-12-10T17:00:00,3,Lombardia,5613,748,6361,80882,87243,-2660,2093,324961,23449,NaN,NaN,435653,4350769,2436786.0,62.0,NaN,NaN
3,2020-12-10T17:00:00,4,Trentino-Alto Adige,670,79,749,12418,13167,-233,461,29324,1356,0.0,0.0,43847,732920,297118.0,2.0,0,NaN
4,2020-12-10T17:00:00,5,Veneto,2606,330,2936,80201,83137,2119,4197,89880,4551,NaN,NaN,177568,2941145,1108040.0,42.0,NaN,NaN
5,2020-12-10T17:00:00,6,Friuli Venezia Giulia,652,62,714,13481,14195,-760,672,22430,1164,NaN,NaN,37789,792239,303968.0,5.0,NaN,NaN
6,2020-12-10T17:00:00,7,Liguria,822,85,907,8051,8958,-289,320,43124,2602,NaN,NaN,54684,637449,301691.0,3.0,i 104903 test antigenici rapidi sono riferiti ...,NaN
7,2020-12-10T17:00:00,8,Emilia-Romagna,2845,229,3074,61643,64717,-1126,1453,68697,6386,NaN,NaN,139800,2276951,1133307.0,17.0,NaN,NaN
8,2020-12-10T17:00:00,9,Toscana,1323,247,1570,20762,22332,-2273,517,85076,3032,NaN,NaN,110440,1680849,993998.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2020-12-10T17:00:00,10,Umbria,310,55,365,5014,5379,-277,182,19738,498,NaN,NaN,25615,447196,234249.0,5.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2020-12-09T17:00:00,1,Piemonte,4102,318,4420,59063,63483,-1030,...,110261,6813,NaN,NaN,180557,1697990,1012701.0,8.0,NaN,NaN
1,1,2020-12-09T17:00:00,2,Valle d'Aosta,90,13,103,753,856,-14,...,5584,343,NaN,NaN,6783,56847,34887.0,4.0,NaN,NaN
2,2,2020-12-09T17:00:00,3,Lombardia,5727,766,6493,83410,89903,-21926,...,320380,23277,NaN,NaN,433560,4326540,2429884.0,24.0,NaN,NaN
3,19,2020-12-09T17:00:00,4,Trentino-Alto Adige,827,78,905,12495,13400,60,...,28647,1339,0.0,0.0,43386,728247,295666.0,1.0,0,NaN
4,3,2020-12-09T17:00:00,5,Veneto,2551,319,2870,78148,81018,1270,...,87950,4403,NaN,NaN,173371,2921028,1101887.0,14.0,NaN,NaN
5,4,2020-12-09T17:00:00,6,Friuli Venezia Giulia,663,63,726,14229,14955,78,...,21040,1122,NaN,NaN,37117,783883,301832.0,5.0,NaN,NaN
6,5,2020-12-09T17:00:00,7,Liguria,841,88,929,8318,9247,-498,...,42526,2591,NaN,NaN,54364,633457,300174.0,4.0,i 99296 test antigenici rapidi sono riferiti a...,NaN
7,6,2020-12-09T17:00:00,8,Emilia-Romagna,2857,233,3090,62753,65843,-679,...,66191,6314,NaN,NaN,138348,2259395,1128691.0,8.0,NaN,NaN
8,7,2020-12-09T17:00:00,9,Toscana,1347,253,1600,23005,24605,-1419,...,82361,2957,NaN,NaN,109923,1671272,990816.0,12.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2020-12-09T17:00:00,10,Umbria,323,56,379,5277,5656,-90,...,19291,486,NaN,NaN,25433,443349,233112.0,2.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2020-12-10T17:00:00,1,Piemonte,4069,310,4379,56728,61107,-2376,974,...,6897,NaN,NaN,181531,1712408,1018261.0,14.0,Il numero dei tamponi antigenici positivi è so...,NaN,4341375
1,2020-12-10T17:00:00,2,Valle d'Aosta,98,10,108,675,783,-73,18,...,344,NaN,NaN,6801,57189,34989.0,0.0,NaN,NaN,125501
2,2020-12-10T17:00:00,3,Lombardia,5613,748,6361,80882,87243,-2660,2093,...,23449,NaN,NaN,435653,4350769,2436786.0,62.0,NaN,NaN,10103969
3,2020-12-10T17:00:00,4,Trentino-Alto Adige,670,79,749,12418,13167,-233,461,...,1356,0.0,0.0,43847,732920,297118.0,2.0,0,NaN,1074819
4,2020-12-10T17:00:00,5,Veneto,2606,330,2936,80201,83137,2119,4197,...,4551,NaN,NaN,177568,2941145,1108040.0,42.0,NaN,NaN,4907704
5,2020-12-10T17:00:00,6,Friuli Venezia Giulia,652,62,714,13481,14195,-760,672,...,1164,NaN,NaN,37789,792239,303968.0,5.0,NaN,NaN,1211357
6,2020-12-10T17:00:00,7,Liguria,822,85,907,8051,8958,-289,320,...,2602,NaN,NaN,54684,637449,301691.0,3.0,i 104903 test antigenici rapidi sono riferiti ...,NaN,1543127
7,2020-12-10T17:00:00,8,Emilia-Romagna,2845,229,3074,61643,64717,-1126,1453,...,6386,NaN,NaN,139800,2276951,1133307.0,17.0,NaN,NaN,4467118
8,2020-12-10T17:00:00,9,Toscana,1323,247,1570,20762,22332,-2273,517,...,3032,NaN,NaN,110440,1680849,993998.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2020-12-10T17:00:00,10,Umbria,310,55,365,5014,5379,-277,182,...,498,NaN,NaN,25615,447196,234249.0,5.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2020-12-10T17:00:00,1,Piemonte,4069,310,4379,56728,61107,-2376,974,...,NaN,NaN,181531,1712408,1018261.0,14.0,Il numero dei tamponi antigenici positivi è so...,NaN,4341375,575
1,2020-12-10T17:00:00,2,Valle d'Aosta,98,10,108,675,783,-73,18,...,NaN,NaN,6801,57189,34989.0,0.0,NaN,NaN,125501,20
2,2020-12-10T17:00:00,3,Lombardia,5613,748,6361,80882,87243,-2660,2093,...,NaN,NaN,435653,4350769,2436786.0,62.0,NaN,NaN,10103969,1036
3,2020-12-10T17:00:00,4,Trentino-Alto Adige,670,79,749,12418,13167,-233,461,...,0.0,0.0,43847,732920,297118.0,2.0,0,NaN,1074819,106
4,2020-12-10T17:00:00,5,Veneto,2606,330,2936,80201,83137,2119,4197,...,NaN,NaN,177568,2941145,1108040.0,42.0,NaN,NaN,4907704,1016
5,2020-12-10T17:00:00,6,Friuli Venezia Giulia,652,62,714,13481,14195,-760,672,...,NaN,NaN,37789,792239,303968.0,5.0,NaN,NaN,1211357,180
6,2020-12-10T17:00:00,7,Liguria,822,85,907,8051,8958,-289,320,...,NaN,NaN,54684,637449,301691.0,3.0,i 104903 test antigenici rapidi sono riferiti ...,NaN,1543127,209
7,2020-12-10T17:00:00,8,Emilia-Romagna,2845,229,3074,61643,64717,-1126,1453,...,NaN,NaN,139800,2276951,1133307.0,17.0,NaN,NaN,4467118,634
8,2020-12-10T17:00:00,9,Toscana,1323,247,1570,20762,22332,-2273,517,...,NaN,NaN,110440,1680849,993998.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2020-12-10T17:00:00,10,Umbria,310,55,365,5014,5379,-277,182,...,NaN,NaN,25615,447196,234249.0,5.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-12-10T17:00:00,5,Veneto,2606,330,2936,80201,83137,2119,4197,...,32.48,2053,1770,20117,6153.0,148,1930,20.86,0.08552,3.62
1,2020-12-10T17:00:00,3,Lombardia,5613,748,6361,80882,87243,-2660,2093,...,72.20,-2528,860,24229,6902.0,172,4581,8.64,0.02071,4.31
2,2020-12-10T17:00:00,12,Lazio,3036,339,3375,87392,90767,-3051,1488,...,40.02,-2975,191,15916,8874.0,68,4471,9.35,0.02537,2.32
3,2020-12-10T17:00:00,8,Emilia-Romagna,2845,229,3074,61643,64717,-1126,1453,...,36.12,-1110,374,17556,4616.0,72,2506,8.28,0.03253,3.13
4,2020-12-10T17:00:00,16,Puglia,1600,200,1800,47002,48802,732,1332,...,54.20,770,415,11168,4265.0,55,545,11.93,0.03323,1.71
5,2020-12-10T17:00:00,15,Campania,1789,140,1929,94236,96165,-1048,1198,...,27.72,-987,-163,14106,10002.0,57,2189,8.49,0.02071,2.94
6,2020-12-10T17:00:00,19,Sicilia,1342,197,1539,35430,36969,-1678,1059,...,33.50,-1645,306,9526,5923.0,32,2705,11.12,0.02131,1.52
7,2020-12-10T17:00:00,1,Piemonte,4069,310,4379,56728,61107,-2376,974,...,53.91,-2335,68,14418,5560.0,84,3266,6.76,0.02244,4.18
8,2020-12-10T17:00:00,6,Friuli Venezia Giulia,652,62,714,13481,14195,-760,672,...,34.44,-748,39,8356,2136.0,42,1390,8.04,0.05547,3.12
9,2020-12-10T17:00:00,9,Toscana,1323,247,1570,20762,22332,-2273,517,...,47.23,-2243,12,9577,3182.0,75,2715,5.40,0.01389,2.97


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,79,27,106,74.53
1,Lombardia,748,288,1036,72.20
2,Marche,86,57,143,60.14
3,Puglia,200,169,369,54.20
4,Piemonte,310,265,575,53.91
5,Valle d'Aosta,10,10,20,50.00
6,Toscana,247,276,523,47.23
7,Umbria,55,62,117,47.01
8,Abruzzo,65,83,148,43.92
9,Liguria,85,124,209,40.67


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,2020-12-08T17:00:00,22,P.A. Trento,426,48,474,1983,2457,-60,149,14241,719,NaN,NaN,17417,398913,142277.0,3.0,NaN,NaN
578,2020-12-09T17:00:00,21,P.A. Bolzano,397,30,427,10457,10884,1,232,14299,604,NaN,NaN,25787,326809,153117.0,0.0,NaN,Il dato sugli ingressi in TI del giorno al mom...
579,2020-12-09T17:00:00,22,P.A. Trento,430,48,478,2038,2516,59,182,14348,735,NaN,NaN,17599,401438,142549.0,1.0,NaN,NaN
580,2020-12-10T17:00:00,21,P.A. Bolzano,239,31,270,10451,10721,-163,244,14698,612,NaN,NaN,26031,328783,153670.0,0.0,NaN,Il dato sugli ingressi in TI del giorno al mom...


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,2020-12-06T17:00:00,4,Trentino-Alto Adige,824,80,904,12434,13338,-293,464,...,8.0,5367.0,1601.0,-304.0,8.85,6.78,8.65,28.98,1.27,-2.15
287,2020-12-07T17:00:00,4,Trentino-Alto Adige,828,81,909,12525,13434,96,320,...,21.0,1481.0,593.0,91.0,8.91,6.77,21.61,53.96,1.25,0.72
288,2020-12-08T17:00:00,4,Trentino-Alto Adige,909,78,987,12353,13340,-94,228,...,23.0,3227.0,1068.0,-172.0,7.90,7.40,7.07,21.35,-3.70,-0.70
289,2020-12-09T17:00:00,4,Trentino-Alto Adige,827,78,905,12495,13400,60,414,...,26.0,4136.0,750.0,142.0,8.62,6.75,10.01,55.20,0.00,0.45


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
